In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import sys
import msgpack as mp
import msgpack_numpy as mpn
sys.path.append(os.path.join(os.getcwd(), ".."))
from support.generate_ar_data import *
from support.pd_support import *
from support.calculations_support import *
from support.mecanum_calculations import *
from support.ar_calculations import *
import pandas as pd
from scipy.signal import savgol_filter
from scipy.interpolate import interp1d

KeyboardInterrupt: 

In [ ]:
_parent_folder = "multi_cam_nov_14"
_base_pth = os.path.dirname(os.getcwd())
_base_pth = os.path.join(_base_pth, "recording_programs", "test_data", _parent_folder)
_base_pth

In [ ]:
""" read excel sheet"""
folder_df = pd.read_excel(os.path.join(_base_pth, "00_analysis", "calibration_files.xlsx"))
folder_df

In [ ]:

def transform_coordinates_1(df, rotmat, org):
    df1 = df.copy( deep=True)
    for i in range(len(df)):
        val = df[["x", "y", "z"]].loc[i].values
        val = np.reshape(val, (3,1))
        _temp = val - org
        translation_correction = np.array([0.045, -0.05, 0.045]).reshape(3, 1)
        _val = rotmat.T @ _temp
        _val = _val - translation_correction
        
        df1["x"].replace({df1["x"].loc[i]: _val[0]}, inplace=True)
        df1["y"].replace({df1["y"].loc[i]: _val[1]}, inplace=True)
        df1["z"].replace({df1["z"].loc[i]: _val[2]}, inplace=True)

    return df1

In [ ]:
mean_err_df = pd.DataFrame(columns=["folder_name", "err_x", "err_y", "err_z", "max_err_x", "max_err_y", "max_err_z", "std_err_x", "std_err_y", "std_err_z"])

In [ ]:
for i in range(len(folder_df)):
    folder_no = i
    #open the calibration files
    _pth = folder_df.folder_name[folder_no]
    _pth = os.path.join(_base_pth, _pth)

    #read the calibration file
    _webcam_calib_folder = folder_df.calibration_folder[folder_no]
    _webcam_calib_folder = os.path.join(_base_pth, _webcam_calib_folder)
    _webcam_calib_pth = os.path.join(_base_pth, _webcam_calib_folder, "webcam_calibration.msgpack")


    with open(_webcam_calib_pth, "rb") as f:
        webcam_calib = mp.Unpacker(f, object_hook=mpn.decode)
        _temp = next(webcam_calib)
        _webcam_cam_mat = _temp[0]
        _webcam_dist = _temp[1]
    _webcam_video_pth = os.path.join(_pth, "webcam_color.msgpack")
    _webcam_timestamp_pth = os.path.join(_pth, "webcam_timestamp.msgpack")


    with open(os.path.join(_webcam_calib_folder, "webcam_rotmat.msgpack"), "rb") as f:
        webcam_rotmat = mp.Unpacker(f, object_hook=mpn.decode)
        _webcam_rot = next(webcam_rotmat)
        _webcam_org = next(webcam_rotmat)

    
    _w_df = get_ar_pose_data(_webcam_video_pth, _webcam_cam_mat, _webcam_dist, is_color=False, single_file=True, flip_frame=False)

    _w_df = add_time_from_file(_w_df, _webcam_timestamp_pth)
    w_df = transform_coordinates_1(_w_df, _webcam_rot, _webcam_org)

    mc_df_pth = os.path.join(_base_pth, folder_df.folder_name[folder_no].split("_")[0], folder_df.folder_name[folder_no]+ ".csv")
    _m_df, st_time = read_df_csv(mc_df_pth)
    _m_df = add_datetime_col(_m_df, st_time, _name="seconds")


    _m_df["x"] = _m_df["lb_x"] + _m_df["rb_x"] + _m_df["lt_x"] + _m_df["rt_x"]
    _m_df["y"] = _m_df["lb_y"] + _m_df["rb_y"] + _m_df["lt_y"] + _m_df["rt_y"]
    _m_df["z"] = _m_df["lb_z"] + _m_df["rb_z"] + _m_df["lt_z"] + _m_df["rt_z"]

    _m_df["x"] = _m_df["x"] / 4
    _m_df["y"] = _m_df["y"] / 4
    _m_df["z"] = _m_df["z"] / 4

    w_df, _m_df = trunkate_dfs(w_df, _m_df)

    w_df = interpolate_target_df(w_df, _m_df)

    error_df = pd.DataFrame()
    error_df["time"] = _m_df["time"]
    error_df["x"] = _m_df["x"] - w_df["x"]
    error_df["y"] = _m_df["y"] - w_df["y"]
    error_df["z"] = _m_df["z"] - w_df["z"]

    mean_err_df = mean_err_df.append({"folder_name": folder_df.folder_name[folder_no], "err_x": error_df["x"].mean(), "err_y": error_df["y"].mean(), "err_z": error_df["z"].mean(), 
                                      "max_err_x": error_df["x"].max(), "max_err_y": error_df["y"].max(), "max_err_z": error_df["z"].max(), 
                                      "std_err_x": error_df["x"].std(), "std_err_y": error_df["y"].std(), "std_err_z": error_df["z"].std()}, ignore_index=True)
    
    print("processed", i)

In [ ]:
_w_df

In [ ]:
plt.plot(_m_df["time"], _m_df["z"], label="mc")
# plt.plot(_w_df["time"], _w_df["z"].rolling(13).median()*-1  +1.5, label="webcam")
plt.plot(w_df["time"], savgol_filter(w_df["z"].values, 51, 1, mode='nearest') + 0.045, label="webcam")
plt.legend()

In [ ]:
# find error between x , y, z and yaw, pitch, roll
error_df = pd.DataFrame()
error_df["time"] = _m_df["time"]
error_df["x"] = _m_df["x"] - w_df["x"]
error_df["y"] = _m_df["y"] - w_df["y"]
error_df["z"] = _m_df["z"] - w_df["z"]

In [ ]:
print(error_df.x.mean())
print(error_df.y.mean())
print(error_df.z.mean())